In [ ]:
import tflearn 
get_ipython().magic(u'matplotlib inline')
import matplotlib.pyplot as plt
from urllib.request import Request, urlopen
import numpy as np
from skimage.util import view_as_windows as vaw
import tensorflow as tf
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)
with tf.Session() as sess:
    print (sess.run(c))
import os, sys
import glob
from progressbar import ProgressBar
from tflearn.data_utils import to_categorical, pad_sequences
from tflearn.layers.recurrent import lstm
from tflearn.layers.core import fully_connected

In [ ]:
# uniprot_keyword = 'keyword1+keyword2+keyword3' # if you want to do multiple search terms
uniprot_keyword = 'antibody+DNA-binding' # what to query uniprot for
string_len = 10 # how many amino acids to take for each substring
uniprot_limit = 1000 # how many proteins to get for each class from uniprot
# how many amino acids to skip during cutting when moving to next cut.
# if this number is 1, it just moves to the next one.
stride = 1

In [ ]:
def install_tensorboard_dep():
  if 'ngrok-stable-linux-amd64.zip' not in os.listdir(os.getcwd()):
    !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
    !unzip ngrok-stable-linux-amd64.zip
    os.system('n')

In [ ]:
def start_tensorboard(d):
  LOG_DIR = d  # tells Tensorboard where to look for log files
  get_ipython().system_raw('tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'.format(LOG_DIR))
  get_ipython().system_raw('./ngrok http 6006 &')
  ! curl -s http://localhost:4040/api/tunnels | python3 -c \
  "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
def get_uniprot_data(kw, numxs):
    '''Goes to the uniprot website and searches for 
       data with the keyword given. Returns the data 
       found up to limit elements.'''

    kws = [kw, 'NOT+' + kw]
    Protein_data = {}
    
    for i in range(2):
        kw = kws[i]
        url1 = 'http://www.uniprot.org/uniprot/?query='
        url2 = '&columns=sequence&format=tab&limit='+str(numxs)
        query_complete = url1 + kw + url2
        request = Request(query_complete)
        response = urlopen(request)
        data = response.read()
        data = str(data, 'utf-8')
        data = data.split('\n')
        data = data[1:-1]
        Protein_data[str(i)] = list(map(lambda x:x.lower(),data))
                     
    x = Protein_data['0'] + Protein_data['1']
    y = np.zeros([len(x), ])
    return x, y

In [ ]:
def cut_strings(seqs, labels, length, padlen=None):
    if padlen is None:
        padlen = int(0.95 * length)

    x = np.zeros([0, length, 1])
    y = np.zeros([0, ])
    count = 0
    xlen = None
    bar = ProgressBar()

    for seq in bar(seqs):
        seq_nums = []
        for letter in seq:
            seq_nums.append(max(ord(letter)-97, 0))

        if len(seq_nums) > length:
            padded_seq = np.pad(np.asarray(seq_nums), (padlen, padlen),
                                          'constant', constant_values=22.)
            cut_seq = vaw(padded_seq, (length, ))
            y = np.concatenate((y, np.ones([cut_seq.shape[0], ])*labels[count]))
            x = np.concatenate((x, cut_seq[..., None]))
            count += 1
        else:
            continue

    print('Used {} proteins.'.format(count))
    plt.hist(x[x != 22.].flatten(), bins=25)
    plt.show()

    return x, y

In [ ]:
# call the get_uniprot_data function to get data and labels
X, Y = get_uniprot_data(uniprot_keyword, uniprot_limit)

In [ ]:
X, Y = cut_strings(X, Y, string_len, stride)

In [ ]:
# get the shape of the data...should be number of examples X length of each sequence
print(X.shape)
print(Y.shape)
print(np.unique(X)) # see all of the different numbers in X

In [ ]:
plt.hist(Y, bins=100)
plt.show()

In [ ]:
Y = tflearn.data_utils.to_categorical(Y, 2)

In [ ]:
 tf.reset_default_graph()

In [ ]:
tb_dir = ('/home/mpcr/DeepProteins')
name = str(string_len) + '_seq_len_'
name += str(stride) + '_stride'
install_tensorboard_dep()
start_tensorboard('tb_dir')

In [ ]:
net_in = tflearn.input_data([None, string_len, 1])
layer1 = lstm(net_in, 10, return_seq=True)
layer2 = lstm(layer1, 10)
out = fully_connected(layer2, 2, activation='softmax')

In [ ]:
net = tflearn.regression(out, optimizer='adam', learning_rate=0.0005,
                         loss='categorical_crossentropy')

In [ ]:
model = tflearn.DNN(net, tensorboard_verbose=2, tensorboard_dir=tb_dir)
model.fit(X, Y, validation_set=0.25, show_metric=True, n_epoch=9,
          batch_size=200, snapshot_step=200, run_id=name)